# Actividad de aprendizaje 4: Programación por objetos

For this case, I used some standars to practice good pracitces:

PEP 8 (identation), PEP 257 (DOCSTRINGS), PEP 318 (DECORATOR), PEP 454 (TYPE HINTS)

## 1. Estás desarrollando un sistema para gestionar una biblioteca digital. Este sistema debe permitir a los usuarios añadir libros, buscar libros por diferentes criterios, tomar prestado libros y gestionar devoluciones.


To make this digiital library work, I would need to create a class for the library, and then create a class for the books.
The library class would have a list of books, and methods to add, remove, lookup, borrow and return books from the list.
The book class would have attributes for title, author, and availability, and method to change the availability of the book.

I would also need to create:
1.    A menu for the user to interact with the library, and a loop to keep the program running until the user decides to exit.
2.    File to store the books in the library, so that the library can be reloaded with the same books when the program is restarted.
3.    A way to save the changes to the library to the file when the program is exited.

In [43]:
from typing import Any, Dict, List, Optional


class Book:
    def __init__(self, *args: Any, **kwargs: Dict[str, Any]) -> None:
        """
        Initialize a new book instance.

        Args:
            title (str): The title of the book.
            author (str): The author of the book.
            year (str): The year the book was published.
            genre (str): The genre of the book.
        """
        self.title = kwargs.get('title', 'Unknown Title')
        self.author = kwargs.get('author', 'Unknown Author')
        self.year = kwargs.get('year', 'Unknown Year')
        self.genre = kwargs.get('genre', 'Unknown Genre')
        self.availability = True

    @property
    def info(self) -> str:
        """
        Return a string representation of the book.

        Returns:
            str: A string representing the book.
        """
        return f'{self.title} by {self.author} ({self.year})'
    
    @property
    def availability(self) -> bool:
        """Get the availability of the book."""
        return self._availability

    @availability.setter
    def availability(self, value: bool) -> None:
        """Set the availability of the book."""
        
        if not isinstance(value, bool):
            raise ValueError("Availability must be a boolean.")
        self._availability = value

    def borrow_book(self) -> bool:
        """
        Mark the book as borrowed.

        Returns:
            bool: True if the book was successfully borrowed, False otherwise.
        """
        try:
            self.availability = False
            return True
        except:
            raise Exception('Book is not available')
        
    def return_book(self) -> bool:
        """
        Mark the book as returned.

        :return: True if the book was successfully returned, False otherwise.
        """
        try:
            self.availability = True
            return True
        except:
            raise Exception('Book is already available')

In [47]:
import sqlite3

class DigitalLibrary:
    def __init__(self, db_name="library.db"):
        self.conn = sqlite3.connect(db_name)
        self.create_table()

    def create_table(self):
        """Create the books table if it doesn't exist."""
        with self.conn:
            self.conn.execute('''
                CREATE TABLE IF NOT EXISTS books (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT NOT NULL,
                    author TEXT NOT NULL,
                    year TEXT NOT NULL,
                    genre TEXT NOT NULL,
                    availability INTEGER NOT NULL
                )
            ''')

    def add_book(self, book):
        """Add a new book to the library."""
        with self.conn:
            self.conn.execute('''
                INSERT INTO books (title, author, year, genre, availability)
                VALUES (?, ?, ?, ?, ?)
            ''', (book.title, book.author, book.year, book.genre, int(book.availability)))

    def lookup_book(self, title):
        """Find a book by title."""
        cursor = self.conn.cursor()
        cursor.execute('SELECT * FROM books WHERE title = ?', (title,))
        return cursor.fetchone()

    def borrow_book(self, title):
        """Mark a book as borrowed."""
        with self.conn:
            cursor = self.conn.cursor()
            cursor.execute('SELECT availability FROM books WHERE title = ?', (title,))
            book = cursor.fetchone()
            if book and book[0] == 1:  # Check if book is available
                cursor.execute('UPDATE books SET availability = 0 WHERE title = ?', (title,))
                return True
        return False

    def return_book(self, title):
        """Mark a book as returned."""
        with self.conn:
            cursor = self.conn.cursor()
            cursor.execute('SELECT availability FROM books WHERE title = ?', (title,))
            book = cursor.fetchone()
            if book and book[0] == 0:  # Check if book is borrowed
                cursor.execute('UPDATE books SET availability = 1 WHERE title = ?', (title,))
                return True
        return False

    def save_library(self):
        """Commit changes to the database."""
        self.conn.commit()

    def load_library(self):
        """Load library from the database (not necessary since we query directly)."""
        pass  # All interactions are done directly with the database

    def display_books(self):
        """Display all books in the library."""
        cursor = self.conn.cursor()
        cursor.execute('SELECT title, author, availability FROM books')
        books = cursor.fetchall()
        for book in books:
            print(f"{book[0]} by {book[1]} - {'Available' if book[2] == 1 else 'Borrowed'}")
    
    def get_book_count(self) -> int:
        """Return the number of books in the library."""
        cursor = self.conn.cursor()
        cursor.execute('SELECT COUNT(*) FROM books')
        count = cursor.fetchone()[0]
        return count

    def main_menu(self):
        """Main menu for library interaction."""
        def show_menu():
            print("1. Add Book")
            print("2. Lookup Book")
            print("3. Borrow Book")
            print("4. Return Book")
            print("5. Display Books")
            print("6. Exit")
        
        show_menu()

        while True:
            choice = input("\nEnter your choice: ")

            if choice == '1':
                title = input("Enter title: ")
                author = input("Enter author: ")
                year = input("Enter year: ")
                genre = input("Enter genre: ")
                book = Book(title=title, author=author, year=year, genre=genre)
                self.add_book(book)
            elif choice == '2':
                title = input("Enter title: ")
                book = self.lookup_book(title)
                if book:
                    print(f"Book found: {book[1]} by {book[2]} - {'Available' if book[5] else 'Borrowed'}")
                else:
                    print("Book not found.")
            elif choice == '3':
                title = input("Enter title: ")
                if self.borrow_book(title):
                    print("Book borrowed successfully.")
                else:
                    print("Book could not be borrowed.")
            elif choice == '4':
                title = input("Enter title: ")
                if self.return_book(title):
                    print("Book returned successfully.")
                else:
                    print("Book could not be returned.")
            elif choice == '5':
                self.display_books()
            elif choice == '6':
                break
            else:
                print("Invalid choice")

    def __del__(self):
        """Close the database connection when the library is destroyed."""
        self.conn.close()


In [48]:
def add_books(library):
    # Create and add 5 books to the library using the Book class
    books = [
        Book(title="To Kill a Mockingbird", author="Harper Lee", year="1960", genre="Fiction"),
        Book(title="1984", author="George Orwell", year="1949", genre="Dystopian"),
        Book(title="The Great Gatsby", author="F. Scott Fitzgerald", year="1925", genre="Fiction"),
        Book(title="The Catcher in the Rye", author="J.D. Salinger", year="1951", genre="Fiction"),
        Book(title="Moby-Dick", author="Herman Melville", year="1851", genre="Adventure"),
    ]

    for book in books:
        library.add_book(book)


if __name__ == "__main__":
    TecLibrary = DigitalLibrary()

    if TecLibrary.get_book_count() < 5:
        add_books(TecLibrary)
    
    TecLibrary.main_menu()

1. Add Book
2. Lookup Book
3. Borrow Book
4. Return Book
5. Display Books
6. Exit
To Kill a Mockingbird by Harper Lee - Available
1984 by George Orwell - Available
The Great Gatsby by F. Scott Fitzgerald - Available
The Catcher in the Rye by J.D. Salinger - Available
Moby-Dick by Herman Melville - Available
Book found: 1984 by George Orwell - Available
Book borrowed successfully.
Book found: 1984 by George Orwell - Borrowed
Book could not be borrowed.
To Kill a Mockingbird by Harper Lee - Available
1984 by George Orwell - Borrowed
The Great Gatsby by F. Scott Fitzgerald - Available
The Catcher in the Rye by J.D. Salinger - Available
Moby-Dick by Herman Melville - Available


## 2.  Estás desarrollando un sistema para gestionar las reservas de mesas en un restaurante. El sistema debe permitir añadir reservas, gestionar el estado de las mesas, buscar reservas por cliente o por fecha, y generar informes sobre el uso de las mesas.

In order to practice but without making it the most efficient possible, I will create as many classes as possible. For this particular case I will create 4:
1.   ReservationSystem: add_reservation(), change_table_status(), search_reservation() and generate_report()
2.   Client
3.   Table
4.   Reservation

In [ ]:
class Client:
    def __init__(self, name: str, phone: str, email: str) -> None:
        self.name = name
        self.phone = phone
        self.email = email
    
    def __str__(self):
        return f"{self.name} ({self.phone}, {self.email})"


In [ ]:
class Table:
    def __init__(self, id, capacity: int) -> None:
        self.id = id
        self.capacity = capacity
        self._is_available = True  # Use _is_available as the internal variable

    @property
    def is_available(self) -> bool:
        return self._is_available  # Return the internal variable

    @is_available.setter
    def is_available(self, value: bool) -> None:
        self._is_available = value  # Set the internal variable

    def update_status(self) -> None:
        """Change the current value to its opposite"""
        self.is_available = not self.is_available
    
    def __str__(self):
        return f"Table {self.number} (Capacity: {self.capacity})"


In [ ]:
class Reservation:
    def __init__(self, client, table, date, time, how_many):
        self.client = client
        self.table = table
        self.date = date
        self.time = time
        self.how_many = how_many
        self.status = "Confirmed"

    def cancel_reservation(self) -> None:
        self.status = "Cancelled"
    
    def update_reservation(self, date, time, how_many) -> None:
        self.date = date
        self.time = time
        self.how_many = how_many
    
    def __str__(self) -> str:
        return f"Reservation for {self.client.name} at {self.table.id} on {self.date} at {self.time} for {self.how_many} people."

In [ ]:
from datetime import datetime
from typing import List

class ReservationSystem:
    """
    A class to manage restaurant reservations, clients, and tables.
    
    Attributes
    ----------
    clients : list
        List of all clients in the system.
    tables : list
        List of all tables available in the restaurant.
    reservations : list
        List of all reservations made.
    """

    def __init__(self):
        """Initialize the reservation system with empty lists for clients, tables, and reservations."""
        self.clients = []
        self.tables = []
        self.reservations = []

    def generate_report(self) -> None:
        """
        Generate a report on the number of occupied tables and reservations for the current day.
        """
        try:

            # Reservations for today
            today_reservation = 0
            today = datetime.now().strftime("%d-%m-%Y")
            for reservation in self.reservations:
                if reservation.date == today and reservation.status == "Confirmed":
                    today_reservation += 1


            # Results
            print("\nRestaurant Report")
            print(f"Today we have: {today_reservation} reservations.")
            self.show_reservations()
        except Exception as e:
            print(f"Error generating report: {e}")

    def add_client(self, client: Client) -> None:
        """
        Add a new client to the system.

        Parameters
        ----------
        client : Client
            The client object to be added.
        """
        try:
            self.clients.append(client)
        except Exception as e:
            print(f"Error adding client: {e}")

    def add_table(self, table: Table) -> None:
        """
        Add a new table to the system.

        Parameters
        ----------
        table : Table
            The table object to be added.
        """
        try:
            self.tables.append(table)
        except Exception as e:
            print(f"Error adding table: {e}")

    def add_reservation(self, reservation: Reservation) -> None:
        """
        Add a new reservation to the system and update the table status.

        Parameters
        ----------
        reservation : Reservation
            The reservation object to be added.
        """
        try:
            self.reservations.append(reservation)
            reservation.table.update_status()
            self.clients.append(reservation.client)
        except Exception as e:
            print(f"Error adding reservation: {e}")

    def cancel_reservation(self, reservation: Reservation) -> None:
        """
        Cancel an existing reservation and update the table status.

        Parameters
        ----------
        reservation : Reservation
            The reservation object to be canceled.
        """
        try:
            reservation.cancel_reservation()
            reservation.table.update_status()
        except Exception as e:
            print(f"Error canceling reservation: {e}")

    def update_reservation(self, reservation: Reservation, date, time, how_many) -> None:
        """
        Update an existing reservation with new details.

        Parameters
        ----------
        reservation : Reservation
            The reservation object to be updated.
        date : str
            The new date for the reservation.
        time : str
            The new time for the reservation.
        how_many : int
            The new number of people for the reservation.
        """
        try:
            reservation.update_reservation(date, time, how_many)
        except Exception as e:
            print(f"Error updating reservation: {e}")

    def show_reservations(self, date: datetime = None) -> None:
        """
        Print all reservations in the system.
        """
        if date == None:
            date = datetime.now().strftime("%d-%m-%Y")

        try:
            for reservation in self.reservations:
                if reservation.status == "Confirmed" and reservation.date == date:
                    print(f'Reservation by {reservation.client.name} at table {reservation.table.id} on {reservation.date} at {reservation.time} for {reservation.how_many} people.')
        except Exception as e:
            print(f"Error showing reservations: {e}")

    def search_reservation(self, filter: str, value: str) -> List:
        """
        Search for all reservations made by a specific client or on a specific date.

        Parameters
        ----------
        filter : str
            The filter type, either "client" or "date".
        value : str
            The client name or date in which reservations are to be searched.

        Returns
        -------
        List
            A list of reservations made by the client or happening on that date.
        """
        try:
            for reservation in self.reservations:
                if (filter == "client" and reservation.client.name == value) or (filter == "date" and reservation.date == value):
                    print(f'Reservation by {reservation.client.name} at table {reservation.table.id} on {reservation.date} at {reservation.time} for {reservation.how_many} people.')
        except Exception as e:
            print(f"Error searching for reservations: {e}")
        
    def __del__(self):
        print("Reservation system closed.")


In [ ]:
from datetime import datetime

# Print a welcome message
print("Welcome to the Reservation System!")
print("Today is:", datetime.now().strftime("%d-%m-%Y"))

# Create a reservation system
system = ReservationSystem()

# Create clients
alice = Client("Alice", "6644423355", "alice@gmail.com")
bob = Client("Bob", "6644423356", "bob@gmail.com")
charlie = Client("Charlie", "6644423357", "charlie@gmail")

# Create tables
table1 = Table(1, 4)
table2 = Table(2, 6)
table3 = Table(3, 2)
table4 = Table(4, 8)

# Add tables to the system
system.add_table(table1)
system.add_table(table2)
system.add_table(table3)
system.add_table(table4)

# Add clients and tables to the system
reservation1 = Reservation(alice, table1, "20-08-2024", "19:00", 4)
reservation2 = Reservation(bob, table2, "20-08-2024", "20:00", 6)
reservation3 = Reservation(charlie, table3, "13-10-2024", "21:00", 2)
reservation4 = Reservation(alice, table4, "20-08-2024", "22:00", 8)
reservation5 = Reservation(bob, table1, "14-10-2024", "19:00", 4)

# Add reservations to the system
system.add_reservation(reservation1)
system.add_reservation(reservation2)
system.add_reservation(reservation3)
system.add_reservation(reservation4)
system.add_reservation(reservation5)

# Generate a report
system.generate_report()

# Cancel a reservation
system.cancel_reservation(reservation2)

# Update a reservation
system.update_reservation(reservation5, "15-10-2024", "19:00", 6)

# Generate a report
system.generate_report()

# Show all reservations
print("\nAll reservations today:")
system.show_reservations()

# Search for reservations by client
print("\nReservations for Alice:")
system.search_reservation("client", "Alice")

# Search for reservations by date
print("\nReservations for 15-10-2024:")
system.search_reservation("date", "15-10-2024")
 

Welcome to the Reservation System!
Today is: 20-08-2024
Reservation system closed.

Restaurant Report
Today we have: 3 reservations.
Reservation by Alice at table 1 on 20-08-2024 at 19:00 for 4 people.
Reservation by Bob at table 2 on 20-08-2024 at 20:00 for 6 people.
Reservation by Alice at table 4 on 20-08-2024 at 22:00 for 8 people.

Restaurant Report
Today we have: 2 reservations.
Reservation by Alice at table 1 on 20-08-2024 at 19:00 for 4 people.
Reservation by Alice at table 4 on 20-08-2024 at 22:00 for 8 people.

All reservations today:
Reservation by Alice at table 1 on 20-08-2024 at 19:00 for 4 people.
Reservation by Alice at table 4 on 20-08-2024 at 22:00 for 8 people.

Reservations for Alice:
Reservation by Alice at table 1 on 20-08-2024 at 19:00 for 4 people.
Reservation by Alice at table 4 on 20-08-2024 at 22:00 for 8 people.

Reservations for 15-10-2024:
Reservation by Bob at table 1 on 15-10-2024 at 19:00 for 6 people.
